In [11]:
import time,serial,random
import numpy as np
import scipy,pybrain3

def servoin(a=90,b=90,c=90,d=90,e=90,f=90,delay=1):
    '''waits end of previous act. and send angles to arduino returns acc-array     
    '''
    while ser.read()!=b'>':
        pass
    acc = ser.readline().split(b'\t')
    A=np.array([[float(acc[0])],[float(acc[1])],[float(acc[2])]])
    print(*A)
    if fall_check(*A)==False:
        data='in'
        for i in a,b,c,d,e,f,delay:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        print(data)
        return True
    else:
        print('I have fallen')
        return False
    
def fall_check(accelx,accely,accelz):
    '''gets measurement of accelerometer
        checks if Robo has fallen
        returns False if Robo - NOT fallen
        returns True if Robo - has fallen'''
    if abs(accelx)>5 or abs(accelz)>5:
        return True
    else: return False
    
def rand():
    '''random angles'''
    servoin(random.randint(0,180),random.randint(0,180),random.randint(0,180),random.randint(0,180),random.randint(0,180),random.randint(0,180))

def bow():
    '''turn forward'''
    servoin(0,180,90,90,90,90,10)
    servoin(90,90,90,90,90,90,1)

def stamp():
    '''walk on place'''
    servoin(125,125,120,135)
    servoin(60,60,55,65)
    
def quatre_step_walk(old):
    '''two support'''
    servoin(b=120,d=120)
    servoin(120,120,120,120)
    time.sleep(1)
    servoin(a=60,c=60)    
    servoin(60,60,60,60)
    old[:4]=[60,60,60,60]
    return old
    
def interpreter(st,old):
    '''gets input string and last value of servoin
        returns tuple of updated values servo
        if str='start' to initials
        if str='a=12 c=13' returns (12,90,13,90,90,90,1)
        if str='(12,90,13,90,90,90,1)' returns the same
        if str='(12,90,13,90)' returns the same
        if str='a=12 c=13 then e=50 then f=40 then start' compile the algorythm
        if str='a=5 c=3 then while a=6 b=2 and a=4 b=4' compile
    '''
    if st=='start':
        old = [90,90,90,90,90,90,1]
        servoin(*old)
        return old
    
    if st=='jackson':
        return quatre_step_walk(old)
    
    if st=='walk':
        old = interpreter('start then for 8 e=80 g=3 and f=80 b=70 a=70 and a=50 and d=70 f=90 e=90 c=70 and e=110 f=110 b=100 a=90 d=90 and c=100 e=120 b=120 g=1 and b=130 d=110 a=120 c=120 and c=140 e=90 f=90 g=3 and e=80 f=80 d=80 b=90 a=60 c=80',old)
          
        
    
    if st[0:5]=='while':
        while True:
            for j in st[6:].split(' and '):
                old=interpreter(j,old)
    
    if st[0:3]=='for':
        for l in range(int(st[4])): 
            for j in st[6:].split(' and '):
                old=interpreter(j,old)
        return old
    
    try: #12,13,15
        q = list(int(i) for  i in st.split(','))
        for j in range(len(q),7): q.append(old[j])
        old = q
        servoin(*old)
        return old
    except ValueError:
        try: #a=12 b=13
            q = old
            tmp = list(st.split())
            for i in tmp: 
                q[ord(str(i).split('=')[0])-97]=int(str(i).split('=')[1])
            old = q
            servoin(*old)
            return old
        except IndexError: #a=12 b=13 then b=15
            for j in st.split(' then '):
                old=interpreter(str(j),old)
            return old
   
if __name__ == "__main__":
    ser = serial.Serial('COM5', 19200, bytesize=8, parity='N', stopbits=1, timeout=2)
    time.sleep(2)
    print("connected to: "+ ser.portstr)
    old_q=[90,90,90,90,90,90,1]
    while True:
        inp=input()
        old_q=interpreter(inp,old_q)
        
    ser.close()


SyntaxError: 'break' outside loop (<ipython-input-11-6bccecf9d669>, line 28)

In [9]:
ser.close()